In [11]:
def evaluate_agent(env, agent,action_type, episodes=100):
    shortfalls = []
    for ep in range(episodes):
        state = env.reset(seed=10000 + ep)
        env.start_transactions()
        done = False
        while not done:
            action, = agent.act(state, add_noise=False)
            state, _, done, info = env.step(action,action_type=action_type)
        shortfalls.append(info.implementation_shortfall)
    return np.array(shortfalls)

def evaluate_ac(env, action_type, episodes=100):
    frac_schedule = env.get_trade_list() / env.total_shares  
    shortfalls = []
    for ep in range(episodes):
        state = env.reset(seed=20000 + ep)
        env.start_transactions()
        done = False
        t = 0
        while not done:
            action = frac_schedule[t]  
            state, _, done, info = env.step(action,action_type)
            t += 1
        shortfalls.append(info.implementation_shortfall)
    return np.array(shortfalls)

In [1]:
import utils
financial_params, ac_params = utils.get_env_param()

In [6]:
import numpy as np
import random
import syntheticChrissAlmgren as sca
from ddpg_agent import Agent

from collections import deque

env1 = sca.MarketEnvironment(price_model='gbm')

agent = Agent(state_size=env1.observation_space_dimension(), action_size=env1.action_space_dimension(), random_seed=0)

lqt = 60

n_trades = 60

tr = 1e-6

episodes = 5000

shortfall_hist = np.array([])
shortfall_deque = deque(maxlen=100)

for episode in range(episodes):
    cur_state = env1.reset(seed = episode, liquid_time = lqt, num_trades = n_trades, lamb = tr)

    env1.start_transactions()

    for i in range(n_trades + 1):

        action = agent.act(cur_state, add_noise = True)

        new_state, reward, done, info = env1.step(action,action_type='rate')

        agent.step(cur_state, action, reward, new_state, done)

        cur_state = new_state

        if info.done:
            shortfall_hist = np.append(shortfall_hist, info.implementation_shortfall)
            shortfall_deque.append(info.implementation_shortfall)
            break

    if (episode + 1) % 100 == 0:
        print('\rEpisode [{}/{}]\tAverage Shortfall: ${:,.2f}'.format(episode + 1, episodes, np.mean(shortfall_deque)))

print('\nAverage Implementation Shortfall: ${:,.2f} \n'.format(np.mean(shortfall_hist)))

Episode [100/5000]	Average Shortfall: $-17,309,879.20
Episode [200/5000]	Average Shortfall: $-11,852,478.54
Episode [300/5000]	Average Shortfall: $-11,988,919.41
Episode [400/5000]	Average Shortfall: $-5,793,870.66
Episode [500/5000]	Average Shortfall: $-15,163,077.18
Episode [600/5000]	Average Shortfall: $-8,436,616.94
Episode [700/5000]	Average Shortfall: $-12,906,645.87
Episode [800/5000]	Average Shortfall: $-6,734,644.69
Episode [900/5000]	Average Shortfall: $-7,921,319.14
Episode [1000/5000]	Average Shortfall: $-18,548,447.35
Episode [1100/5000]	Average Shortfall: $-10,811,057.37
Episode [1200/5000]	Average Shortfall: $-7,460,998.41
Episode [1300/5000]	Average Shortfall: $-9,478,452.63
Episode [1400/5000]	Average Shortfall: $-8,218,781.51
Episode [1500/5000]	Average Shortfall: $-6,378,010.28
Episode [1600/5000]	Average Shortfall: $-15,518,798.74
Episode [1700/5000]	Average Shortfall: $-7,130,888.45
Episode [1800/5000]	Average Shortfall: $-9,845,493.97
Episode [1900/5000]	Average S

In [12]:
agent_sfs = evaluate_agent(env1, agent, episodes=1000,action_type='rate')
ac_sfs    = evaluate_ac(env1, episodes=1000,action_type='rate')

print("Agent: mean=${:.2f}, std=${:.2f}".format(agent_sfs.mean(), agent_sfs.std()))
print("AC   : mean=${:.2f}, std=${:.2f}".format(ac_sfs.mean(),    ac_sfs.std()))

Agent: mean=$-18549372.85, std=$62746912.10
AC   : mean=$-20787826.83, std=$68740586.02


In [14]:
import numpy as np
import random
import syntheticChrissAlmgren as sca
from ddpg_agent import Agent

from collections import deque

env2 = sca.MarketEnvironment(price_model='gbm')

agent = Agent(state_size=env2.observation_space_dimension(), action_size=env2.action_space_dimension(), random_seed=0)

lqt = 60

n_trades = 60

tr = 1e-6

episodes = 5000

shortfall_hist = np.array([])
shortfall_deque = deque(maxlen=100)

for episode in range(episodes):
    cur_state = env2.reset(seed = episode, liquid_time = lqt, num_trades = n_trades, lamb = tr)

    env2.start_transactions()

    for i in range(n_trades + 1):

        action = agent.act(cur_state, add_noise = True)

        new_state, reward, done, info = env2.step(action,action_type='percent')

        agent.step(cur_state, action, reward, new_state, done)

        cur_state = new_state

        if info.done:
            shortfall_hist = np.append(shortfall_hist, info.implementation_shortfall)
            shortfall_deque.append(info.implementation_shortfall)
            break

    if (episode + 1) % 100 == 0:
        print('\rEpisode [{}/{}]\tAverage Shortfall: ${:,.2f}'.format(episode + 1, episodes, np.mean(shortfall_deque)))

print('\nAverage Implementation Shortfall: ${:,.2f} \n'.format(np.mean(shortfall_hist)))

Episode [100/5000]	Average Shortfall: $2,076,750.81
Episode [200/5000]	Average Shortfall: $1,994,787.81
Episode [300/5000]	Average Shortfall: $2,103,292.56
Episode [400/5000]	Average Shortfall: $1,875,205.12
Episode [500/5000]	Average Shortfall: $1,441,223.10
Episode [600/5000]	Average Shortfall: $1,255,380.59
Episode [700/5000]	Average Shortfall: $1,222,341.08
Episode [800/5000]	Average Shortfall: $1,328,063.60
Episode [900/5000]	Average Shortfall: $1,340,674.54
Episode [1000/5000]	Average Shortfall: $1,335,749.10
Episode [1100/5000]	Average Shortfall: $1,224,447.74
Episode [1200/5000]	Average Shortfall: $1,206,118.75
Episode [1300/5000]	Average Shortfall: $1,180,698.82
Episode [1400/5000]	Average Shortfall: $1,209,347.68
Episode [1500/5000]	Average Shortfall: $1,191,000.85
Episode [1600/5000]	Average Shortfall: $1,193,991.03
Episode [1700/5000]	Average Shortfall: $1,195,971.65
Episode [1800/5000]	Average Shortfall: $1,177,017.75
Episode [1900/5000]	Average Shortfall: $1,219,652.78
Ep

In [16]:
agent_sfs = evaluate_agent(env2, agent, episodes=2000,action_type='percent')
ac_sfs    = evaluate_ac(env2, episodes=1000,action_type='percent')

print("Agent: mean=${:.2f}, std=${:.2f}".format(agent_sfs.mean(), agent_sfs.std()))
print("AC   : mean=${:.2f}, std=${:.2f}".format(ac_sfs.mean(),    ac_sfs.std()))

Agent: mean=$1216947.13, std=$541780.22
AC   : mean=$701844.59, std=$494452.07


In [18]:
import numpy as np
import random
import syntheticChrissAlmgren as sca
from ddpg_agent import Agent

from collections import deque

env3 = sca.MarketEnvironment(price_model='gbm')

agent = Agent(state_size=env3.observation_space_dimension(), action_size=env3.action_space_dimension(), random_seed=0)

lqt = 60

n_trades = 60

tr = 1e-6

episodes = 5000

shortfall_hist = np.array([])
shortfall_deque = deque(maxlen=100)

for episode in range(episodes):
    cur_state = env3.reset(seed = episode, liquid_time = lqt, num_trades = n_trades, lamb = tr)

    env3.start_transactions()

    for i in range(n_trades + 1):

        action = agent.act(cur_state, add_noise = True)

        new_state, reward, done, info = env3.step(action,action_type='volume')

        agent.step(cur_state, action, reward, new_state, done)

        cur_state = new_state

        if info.done:
            shortfall_hist = np.append(shortfall_hist, info.implementation_shortfall)
            shortfall_deque.append(info.implementation_shortfall)
            break

    if (episode + 1) % 100 == 0:
        print('\rEpisode [{}/{}]\tAverage Shortfall: ${:,.2f}'.format(episode + 1, episodes, np.mean(shortfall_deque)))

print('\nAverage Implementation Shortfall: ${:,.2f} \n'.format(np.mean(shortfall_hist)))

Episode [100/5000]	Average Shortfall: $-497,635.83
Episode [200/5000]	Average Shortfall: $830,100.55
Episode [300/5000]	Average Shortfall: $526,332.94
Episode [400/5000]	Average Shortfall: $1,319,907.88
Episode [500/5000]	Average Shortfall: $-3,981,736.91
Episode [600/5000]	Average Shortfall: $-3,507,205.56
Episode [700/5000]	Average Shortfall: $-5,371,893.55
Episode [800/5000]	Average Shortfall: $-1,969,770.22
Episode [900/5000]	Average Shortfall: $-2,335,571.93
Episode [1000/5000]	Average Shortfall: $-7,576,319.91
Episode [1100/5000]	Average Shortfall: $-4,745,126.95
Episode [1200/5000]	Average Shortfall: $-2,922,867.33
Episode [1300/5000]	Average Shortfall: $-3,628,673.17
Episode [1400/5000]	Average Shortfall: $-2,677,349.59
Episode [1500/5000]	Average Shortfall: $-2,177,595.63
Episode [1600/5000]	Average Shortfall: $417,881.66
Episode [1700/5000]	Average Shortfall: $675,854.35
Episode [1800/5000]	Average Shortfall: $686,262.80
Episode [1900/5000]	Average Shortfall: $565,579.86
Epis

In [19]:
agent_sfs = evaluate_agent(env1, agent, episodes=1000,action_type='volume')
ac_sfs    = evaluate_ac(env1, episodes=1000,action_type='volume')

print("Agent: mean=${:.2f}, std=${:.2f}".format(agent_sfs.mean(), agent_sfs.std()))
print("AC   : mean=${:.2f}, std=${:.2f}".format(ac_sfs.mean(),    ac_sfs.std()))

Agent: mean=$-7116361.59, std=$29858595.79
AC   : mean=$-20159259.87, std=$66477810.11


In [20]:
import numpy as np
import random
import syntheticChrissAlmgren as sca
from ddpg_agent import Agent

from collections import deque

env4 = sca.MarketEnvironment(price_model='gbm')

agent = Agent(state_size=env4.observation_space_dimension(), action_size=env4.action_space_dimension(), random_seed=0)

lqt = 60

n_trades = 60

tr = 1e-6

episodes = 5000

shortfall_hist = np.array([])
shortfall_deque = deque(maxlen=100)

for episode in range(episodes):
    cur_state = env4.reset(seed = episode, liquid_time = lqt, num_trades = n_trades, lamb = tr)

    env4.start_transactions()

    for i in range(n_trades + 1):

        action = agent.act(cur_state, add_noise = True)

        new_state, reward, done, info = env4.step(action,action_type='time')

        agent.step(cur_state, action, reward, new_state, done)

        cur_state = new_state

        if info.done:
            shortfall_hist = np.append(shortfall_hist, info.implementation_shortfall)
            shortfall_deque.append(info.implementation_shortfall)
            break

    if (episode + 1) % 100 == 0:
        print('\rEpisode [{}/{}]\tAverage Shortfall: ${:,.2f}'.format(episode + 1, episodes, np.mean(shortfall_deque)))

print('\nAverage Implementation Shortfall: ${:,.2f} \n'.format(np.mean(shortfall_hist)))

Episode [100/5000]	Average Shortfall: $2,505,160.50
Episode [200/5000]	Average Shortfall: $2,336,543.28
Episode [300/5000]	Average Shortfall: $1,605,645.45
Episode [400/5000]	Average Shortfall: $1,493,499.99
Episode [500/5000]	Average Shortfall: $1,663,269.33
Episode [600/5000]	Average Shortfall: $1,672,059.14
Episode [700/5000]	Average Shortfall: $1,652,378.48
Episode [800/5000]	Average Shortfall: $1,640,728.76
Episode [900/5000]	Average Shortfall: $1,625,346.09
Episode [1000/5000]	Average Shortfall: $1,628,674.95
Episode [1100/5000]	Average Shortfall: $1,626,240.34
Episode [1200/5000]	Average Shortfall: $1,629,713.82
Episode [1300/5000]	Average Shortfall: $1,629,838.24
Episode [1400/5000]	Average Shortfall: $1,643,904.11
Episode [1500/5000]	Average Shortfall: $1,623,959.37
Episode [1600/5000]	Average Shortfall: $1,633,592.27
Episode [1700/5000]	Average Shortfall: $1,638,937.90
Episode [1800/5000]	Average Shortfall: $1,647,239.10
Episode [1900/5000]	Average Shortfall: $1,640,114.12
Ep

In [21]:
agent_sfs = evaluate_agent(env4, agent, episodes=1000,action_type='time')
ac_sfs    = evaluate_ac(env4, episodes=1000,action_type='time')

print("Agent: mean=${:.2f}, std=${:.2f}".format(agent_sfs.mean(), agent_sfs.std()))
print("AC   : mean=${:.2f}, std=${:.2f}".format(ac_sfs.mean(),    ac_sfs.std()))

Agent: mean=$1108842.91, std=$52203.74
AC   : mean=$-7799673.85, std=$25611026.98


In [22]:
import numpy as np
import random
import syntheticChrissAlmgren as sca
from ddpg_agent import Agent

from collections import deque

env5 = sca.MarketEnvironment(price_model='gbm')

agent = Agent(state_size=env5.observation_space_dimension(), action_size=env5.action_space_dimension(), random_seed=0)

lqt = 60

n_trades = 60

tr = 1e-6

episodes = 5000

shortfall_hist = np.array([])
shortfall_deque = deque(maxlen=100)

for episode in range(episodes):
    cur_state = env5.reset(seed = episode, liquid_time = lqt, num_trades = n_trades, lamb = tr)

    env5.start_transactions()

    for i in range(n_trades + 1):

        action = agent.act(cur_state, add_noise = True)

        new_state, reward, done, info = env5.step(action,action_type='volatility')

        agent.step(cur_state, action, reward, new_state, done)

        cur_state = new_state

        if info.done:
            shortfall_hist = np.append(shortfall_hist, info.implementation_shortfall)
            shortfall_deque.append(info.implementation_shortfall)
            break

    if (episode + 1) % 100 == 0:
        print('\rEpisode [{}/{}]\tAverage Shortfall: ${:,.2f}'.format(episode + 1, episodes, np.mean(shortfall_deque)))

print('\nAverage Implementation Shortfall: ${:,.2f} \n'.format(np.mean(shortfall_hist)))

Episode [100/5000]	Average Shortfall: $-680,193.67
Episode [200/5000]	Average Shortfall: $589,922.93
Episode [300/5000]	Average Shortfall: $608,024.18
Episode [400/5000]	Average Shortfall: $591,746.95
Episode [500/5000]	Average Shortfall: $143,616.76
Episode [600/5000]	Average Shortfall: $-607,543.63
Episode [700/5000]	Average Shortfall: $-846,352.60
Episode [800/5000]	Average Shortfall: $-265,621.43
Episode [900/5000]	Average Shortfall: $-484,828.52
Episode [1000/5000]	Average Shortfall: $-1,152,712.08
Episode [1100/5000]	Average Shortfall: $-601,284.06
Episode [1200/5000]	Average Shortfall: $-369,757.61
Episode [1300/5000]	Average Shortfall: $-109,158.96
Episode [1400/5000]	Average Shortfall: $-526,430.01
Episode [1500/5000]	Average Shortfall: $-277,126.55
Episode [1600/5000]	Average Shortfall: $-540,603.17
Episode [1700/5000]	Average Shortfall: $-448,559.96
Episode [1800/5000]	Average Shortfall: $-367,422.86
Episode [1900/5000]	Average Shortfall: $-525,526.78
Episode [2000/5000]	Ave

In [23]:
agent_sfs = evaluate_agent(env5, agent, episodes=1000,action_type='volatility')
ac_sfs    = evaluate_ac(env5, episodes=1000,action_type='volatility')

print("Agent: mean=${:.2f}, std=${:.2f}".format(agent_sfs.mean(), agent_sfs.std()))
print("AC   : mean=${:.2f}, std=${:.2f}".format(ac_sfs.mean(),    ac_sfs.std()))

Agent: mean=$566685.58, std=$1124741.22
AC   : mean=$-19342932.99, std=$63414249.59


In [24]:
import numpy as np
import random
import syntheticChrissAlmgren as sca
from ddpg_agent import Agent

from collections import deque

env6 = sca.MarketEnvironment(price_model='gbm')

agent = Agent(state_size=env6.observation_space_dimension(), action_size=env6.action_space_dimension(), random_seed=0)

lqt = 60

n_trades = 60

tr = 1e-6

episodes = 5000

shortfall_hist = np.array([])
shortfall_deque = deque(maxlen=100)

for episode in range(episodes):
    cur_state = env6.reset(seed = episode, liquid_time = lqt, num_trades = n_trades, lamb = tr)

    env6.start_transactions()

    for i in range(n_trades + 1):

        action = agent.act(cur_state, add_noise = True)

        new_state, reward, done, info = env6.step(action,action_type='spread')

        agent.step(cur_state, action, reward, new_state, done)

        cur_state = new_state

        if info.done:
            shortfall_hist = np.append(shortfall_hist, info.implementation_shortfall)
            shortfall_deque.append(info.implementation_shortfall)
            break

    if (episode + 1) % 100 == 0:
        print('\rEpisode [{}/{}]\tAverage Shortfall: ${:,.2f}'.format(episode + 1, episodes, np.mean(shortfall_deque)))

print('\nAverage Implementation Shortfall: ${:,.2f} \n'.format(np.mean(shortfall_hist)))

Episode [100/5000]	Average Shortfall: $2,457,708.68
Episode [200/5000]	Average Shortfall: $2,420,869.49
Episode [300/5000]	Average Shortfall: $1,560,278.73
Episode [400/5000]	Average Shortfall: $1,189,394.88
Episode [500/5000]	Average Shortfall: $1,144,464.15
Episode [600/5000]	Average Shortfall: $1,114,771.87
Episode [700/5000]	Average Shortfall: $1,128,030.01
Episode [800/5000]	Average Shortfall: $1,219,939.13
Episode [900/5000]	Average Shortfall: $1,442,734.09
Episode [1000/5000]	Average Shortfall: $1,524,400.56
Episode [1100/5000]	Average Shortfall: $1,340,523.42
Episode [1200/5000]	Average Shortfall: $1,272,795.19
Episode [1300/5000]	Average Shortfall: $1,214,205.01
Episode [1400/5000]	Average Shortfall: $1,185,637.20
Episode [1500/5000]	Average Shortfall: $1,138,258.27
Episode [1600/5000]	Average Shortfall: $1,185,506.52
Episode [1700/5000]	Average Shortfall: $1,277,346.65
Episode [1800/5000]	Average Shortfall: $1,393,893.83
Episode [1900/5000]	Average Shortfall: $1,549,886.99
Ep

In [25]:
agent_sfs = evaluate_agent(env6, agent, episodes=1000,action_type='spread')
ac_sfs    = evaluate_ac(env6, episodes=1000,action_type='spread')

print("Agent: mean=${:.2f}, std=${:.2f}".format(agent_sfs.mean(), agent_sfs.std()))
print("AC   : mean=${:.2f}, std=${:.2f}".format(ac_sfs.mean(),    ac_sfs.std()))

Agent: mean=$1275223.56, std=$32632.15
AC   : mean=$-7298779.76, std=$24061965.96
